In [ ]:
import mapsims
import numpy as np
import healpy as hp
import pysm3.units as u
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt

# Running on the fly simulations with mapsims

The `mapsims` package allows to generate maps on-the-fly, so they can be generated as needed inside other pipelines.

The interface is the `SONoiseSimulator` class, specify `nside` to work with HEALPix maps, `shape` and `wcs` for CAR (see lesson 6).

The class constructor just initializes the configuration of the class, doesn't load neither spectra nor hitmaps.

In [ ]:
mapsims.SONoiseSimulator?

In [ ]:
noise_sim = mapsims.SONoiseSimulator(nside=128)

The `simulate` method does all the processing, it loads the HEALPix or CAR hitmaps (from a path when run at NERSC or downloading and caching them locally) and calls the `so_noise_models` package to generate the expected spectra given the instrument configuration.

The hitmaps were generated in time domain executing parallel runs with TOAST and then saved to disk. They are only used as a relative weighting, the global noise properties are driven by the spectra from `so_noise_models`.

The `simulate` method gets the expected spectra, weights them by the sky fraction and then generates isotropic noise over the whole sky with `synfast` for HEALPix and `sym_expand` for CAR and then scales it by the relative hitmap.

It always simulates a dichroic tube at a time so that also the cross-correlation between the channels is simulated and captured in the output maps.

In [ ]:
noise_sim.simulate?

In [ ]:
tube = "ST1"

In [ ]:
noise_maps = noise_sim.simulate(tube)

In [ ]:
noise_maps.shape

In [ ]:
channels = mapsims.parse_channels("tube:" + tube)[0]

In [ ]:
channels

In [ ]:
for ch, m in zip(channels, noise_maps):
    hp.mollview(m[0][1], title="EE map " + ch.tag, unit="uK", min=-10, max=10)

## Compare cross-spectra

We can take a cross-spectrum between the maps of the 2 tubes and compare with the expected spectrum from `noise.get_fullsky_noise_spectra`.

**TODO**

## Simulate multiple tubes

A single `SONoiseSimulator` object is capable of simulating noise for different tubes. See for example a loop to simulate all SAT channels.

In [ ]:
tubes = ["ST{}".format(i) for i in range(4)]

In [ ]:
for tube in tubes:
    noise_maps = noise_sim.simulate(tube)
    for ch, m in zip(mapsims.parse_channels("tube:" + tube)[0], noise_maps):
        hp.mollview(m[0][1], title="EE map " + ch.tag, unit="$uK$", min=-10, max=10)

## Generate multiple splits

Override `nsplits` to generate multiple splits, the output maps will be a 3 dimensional array where for each split we have the 3 IQU components.

In [ ]:
noise_maps_splits = noise_sim.simulate("ST0", nsplits=4)

In [ ]:
noise_maps_splits.shape